Brusselator for exluded volume in 1D

In [ ]:
import numpy as np
from scipy.fft import fftn, ifftn, fftfreq
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import rc
import matplotlib.animation as animation
%matplotlib inline


def create_initial_array(num_of_nodes, ampl = 0.1):
        return np.random.rand(num_of_nodes) * ampl
    #return initial concentration of substance. Can be random, should be positive


class Brusselator():
    def __init__(self, N = 100, L = 40*np.pi, a=1.5, b=2.34, D0=2.8, D1=22.4, NUM_STEPS=10**4, dt = 0.005, method = 'pseudo-spectral'):
        self.N = N
        self.L = L
        self.a = a
        self.b = b
        self.D0 = D0
        self.D1 = D1
        self.NUM_STEPS = NUM_STEPS
        self.dt = dt
        fu = lambda u,v: self.a + (u**2)*v - (self.b+1)*u
        fv = lambda u,v: self.b*u-(u**2)*v
        self.fu = fu
        self.fv = fv
        self.method = method
# code for initializing data, so that the same model can be run for different methods.
        base_state_u = self.a
        base_state_v = self.b/self.a
        # self.initial_data_u = create_initial_array(self.N)+base_state_u
        self.initial_data_u = create_initial_array(self.N)+base_state_u        
        self.initial_data_v = create_initial_array(self.N)+base_state_v

    def print_parameters(self):
        return f"Simulation for a={self.a},b={self.b},NUM_STEPS={self.NUM_STEPS},N={self.N}, D0={self.D0}, D1={self.D1}, dt = {self.dt}, L={self.L}"

In [ ]:
def create_wavenumber_array(num_modes, L):
    # create array for wavenumbers in scipy format.
    num_modes = num_modes - num_modes%2 # make num of nodes even. Why?
    # this causes the program not to work for odd num_nodes
    # FIX THIS!
    wavenum_array = np.zeros(num_modes)
    wavenum = 2*np.pi/L
    p1 = np.array([(wavenum * i) for i in range(int(num_modes/2))])
    p2 = np.array([wavenum * (-num_modes+n) for n in range(int(num_modes/2), num_modes)])
    wavenum_array = np.concatenate((p1,p2))
    return wavenum_array

def create_time_operator(wavenums, diffusion_coeff, timestep, two_dim = True):
    ## create a time evolution operator
    wavenums = wavenums**2
    return np.exp(-(wavenums)*timestep*diffusion_coeff)
def apply_step_fourier(u,operator):
        fft_u = fftn(u)
        fft_u = fft_u*operator
        return ifftn(fft_u).real

## Von Neumann boundary conditions
def apply_boundary_condition(Z):
    Z[0, :] = Z[1, :]
    Z[-1, :] = Z[-2, :]
    Z[:, 0] = Z[:, 1]
    Z[:, -1] = Z[:, -2]
    return Z